Top 3 Apps finalised which also have reviews:  
- AMAZON SHOPPING  
- GMAIL  
- COOKING FEVER  

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
reviews_data = pd.read_csv('googleplaystore_user_reviews.csv')
apps_to_look_for = ['Amazon Shopping','Gmail','Cooking Fever']
reviews_data.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [15]:
# Creating a Function to Fetch reviews and append in one dataframe!
def fetch_reviews_df_apps(df,list_of_apps):
    final_df = pd.DataFrame(columns= df.columns)
    for i in list_of_apps:
        if i in df.App.get_values():
            sub_df = df[df.App == i].reset_index(drop=True)
        else:
            pass
        # Merging/Concatenating DF's
        final_df = pd.concat([final_df,sub_df])
    # Dropping the NA values rows
    final_df.dropna(axis='rows',inplace=True)
    return final_df.reset_index(drop=True)

In [16]:
prepared_reviews_df = fetch_reviews_df_apps(reviews_data,apps_to_look_for)
prepared_reviews_df.App.value_counts()

Cooking Fever      135
Gmail               64
Amazon Shopping     45
Name: App, dtype: int64

In [17]:
prepared_reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,Amazon Shopping,Terrible change way switch wishlists. When scr...,Negative,-0.166667,0.700000
1,Amazon Shopping,"Completely useless. They changed triple price,...",Negative,-0.300000,0.600000
2,Amazon Shopping,"Hey, Jeff Bezos, employees living cars feeling...",Positive,0.100000,0.657778
3,Amazon Shopping,"Good app, keeps supposed to. Error messages do...",Positive,0.875000,0.600000
4,Amazon Shopping,This actually looking dogs error messages unde...,Neutral,0.000000,0.100000


In [19]:
# Let's finalise 0,1,2 for Negative/Positive/Neutral Sentiments

prepared_reviews_df['Sentiment'] = prepared_reviews_df['Sentiment'].apply(lambda x: 0 if x == 'Negative' else(1 if x == 'Positive' else 2))
prepared_reviews_df.head(10)

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,Amazon Shopping,Terrible change way switch wishlists. When scr...,0,-0.166667,0.700000
1,Amazon Shopping,"Completely useless. They changed triple price,...",0,-0.300000,0.600000
2,Amazon Shopping,"Hey, Jeff Bezos, employees living cars feeling...",1,0.100000,0.657778
3,Amazon Shopping,"Good app, keeps supposed to. Error messages do...",1,0.875000,0.600000
4,Amazon Shopping,This actually looking dogs error messages unde...,2,0.000000,0.100000
5,Amazon Shopping,Amazon stretched thin? The slow hell. Click it...,0,-0.100000,0.475000
6,Amazon Shopping,"Once unnecessary permissions disabled, mostly ...",0,-0.180000,0.600000
7,Amazon Shopping,"Always slow painful use, I feel back days dial...",2,0.000000,0.512500
8,Amazon Shopping,Why Apple's supports long press shortcuts not?...,1,0.008333,0.533333
9,Amazon Shopping,The home page sloppy otherwise connects many A...,0,-0.013889,0.666667


In [20]:
sentiment_data = prepared_reviews_df.loc[:,['App','Translated_Review','Sentiment']]
sentiment_data.head(8)

,App,Translated_Review,Sentiment
0,Amazon Shopping,Terrible change way switch wishlists. When scr...,0
1,Amazon Shopping,"Completely useless. They changed triple price,...",0
2,Amazon Shopping,"Hey, Jeff Bezos, employees living cars feeling...",1
3,Amazon Shopping,"Good app, keeps supposed to. Error messages do...",1
4,Amazon Shopping,This actually looking dogs error messages unde...,2
5,Amazon Shopping,Amazon stretched thin? The slow hell. Click it...,0
6,Amazon Shopping,"Once unnecessary permissions disabled, mostly ...",0
7,Amazon Shopping,"Always slow painful use, I feel back days dial...",2


### Now from here, we will do the following actions on the Translated_Review column:  
- Remove Punctuations  
- Remove unwanted STOPWORDS  
- Remove **expressions/symbols like commas,questions marks, apostrophies, dash, etc.**  
- Lemmatization  
- Prepare the Tokenization/Word Embedding for the Review.  